In [1]:
# Important! This notebook runs on Python 3

import pandas as pd
import numpy as np
from numpy import random

In [2]:
# Necessary world information

# Prices and Costs
# Prices of one beer at each level of the supply chain.
retail_price = 10
wholesale_price = 9
regional_warehouse_price = 8
factory_price = 7
field_price = 6
# Cost of holding one beer during one day on warehouse.
# Assumed to be the same for all levels
warehouse_price = 0.5
# Cost of backlog: non fulfilled orders
backlog_cost = 0.5

# Initial Inventories
retail_ininv = 100
wholesale_ininv = 100
regional_warehouse_ininv = 100
factory_ininv = 100

# Necessary customer demand and field supply trends
# Customer demand will be added a small random effect each time
# Field will stay non-random
#customer_demand = pd.read_csv(beer_customer_demand.csv)
#field_supply = pd.read_csv(beer_field_supply.csv)

In [9]:
class Customer:
    """
    This type of agent doesn't learn, just interacts with Retail by demanding beer
    """
    def __init__(demand_trend):
        self.demand = demand_trend

class Fields:
    """
    This type of agent doesn't learn, just interacts with Factory by supplying beer
    """
    def __init__(supply_trend):
        self.supply = supply_trend    

class Agent:
    """
    Creates a Beer Supply Chain Agent ready to start interacting
    with other agents and learn.
    input:
    * name (string) indicating the type of agent, can be one of four:
    {Retail,Wholesale,Regional_Warehouse,Factory} 
    * inventory (numeric) starting inventory at day 1 
    output: an object of type Agent
    """
    def __init__(self,name,inventory):
        
        # I am letting different levels have different selling and buying prices
        # This could also include different warehousing/backlogs costs
        if name == "Retail":
            self.selling_price = retail_price
            self.buying_price = wholesale_price
            self.downstream_agent =  ""# customer (fixed, not an agent)
            self.upstream_agent =  ""# wholesale
        elif name == "Wholesale":
            self.selling_price = wholesale_price
            self.buying_price = regional_warehouse_price
            self.downstream_agent =  ""# retail
            self.upstream_agent =  ""# regional_warehouse
        elif name == "Regional_Warehouse":
            self.selling_price = regional_warehouse_price
            self.buying_price = factory_price
            self.downstream_agent =  ""# wholesale
            self.upstream_agent =  ""# factory
        elif name == "Factory":
            self.selling_price = factory_price
            self.buying_price = field_price
            self.downstream_agent =  ""# regional_warehouse
            self.upstream_agent =  ""# fields (fixed, not an agent)
        
        self.name = name
        self.inventory = inventory
        self.total_warehousing_costs = 0
        self.total_money = 0
        self.backlog = 0
        self.current_policy = []
        self.current_payout = []
        self.best_policy = [0] * 365
        self.best_payout = [0] * 365
    
    def pay_for_warehousing(self):
        # Pays for warehousing of inventory: must be done either
        # "first thing in the morning" or "last time in the night"
        self.total_money = self.total_money - \
                self.inventory * warehouse_price
    
    def receive_upstream(self,orders):
        # Receives orders from upstream agent first thing in the morning
        self.inventory = self.inventory + orders
        self.total_money = self.total_money - \
                orders * self.buying_price
        
    def give_downstream(self,orders):
        # Checks if he has availability to fulfill order,
        # fulfills as much as he can
        if self.inventory >= orders:
            self.total_money = self.total_money + \
                orders * self.selling_price
            self.inventory = self.inventory - orders
            return orders
        else:
            orders_that_could_be_fulfilled = self.inventory
            # Sells all its inventory
            self.total_money = self.total_money + \
                self.inventory * self.selling_price
            # If there were non fulfilled orders, those cause a penalty
            self.backlog = (orders - self.inventory) * backlog_cost
            self.inventory = 0
            return orders_that_could_be_fulfilled

In [10]:
# Creating Supply Chain Agents
customer_agent = Customer(customer_demand)
retail_agent = Agent("Retail", retail_ininv)
wholesale_agent = Agent("Wholesale", wholesale_ininv)
regional_warehouse_agent = Agent("Regional_Warehouse", regional_warehouse_ininv)
factory_agent = Agent("Factory",factory_ininv)
fields_agent = Fields(field_supply)

# Assigning interactions
retail_agent.downstream_agent = customer_agent
retail_agent.upstream_agent = wholesale_agent
wholesale_agent.downstream_agent = retail_agent
wholesale_agent.upstream_agent = regional_warehouse_agent
regional_warehouse_agent.downstream_agent = wholesale_agent
regional_warehouse_agent.upstream_agent = factory_agent
factory_agent.downstream_agent = regional_warehouse_agent
factory_agent.upstream_agent = fields_agent

NameError: name 'customer' is not defined

In [6]:
# THIS CELL IS HARDCODED BUT TRENDS COME FROM A CSV

# One day of transactions
# Hardcoding these since I'm only testing the functions on day 1 - need to create a seasonality csv for them
customer_demand = 51
field_supply = 60

# AND DEMANDS ARE WHAT THE AGENTS WANT TO LEARN. HARCODING TO TRY OUT FUNCTIONS BUT THIS IS NOT HOW IT WILL WORK

# Demands come from time t-1
retail_demand = customer_demand  # hardcoded, just a madeup number
wholesale_demand = retail_demand + 25  # hardcoded, just a madeup number
regional_warehouse_demand = wholesale_demand + 25  # hardcoded, just a madeup number
factory_demand = regional_warehouse_demand - 25  # hardcoded, just a madeup number

In [ ]:
# Everyone pays for warehousing overnight
factory_agent.pay_for_warehousing
regional_warehouse_agent.pay_for_warehousing
wholesale_agent.pay_for_warehousing
retail_agent.pay_for_warehousing

# Orders are fulfilled first time in the morning
# Everyone gets their shippings at the same time

fulfilled_to_factory = field_supply
factory_agent.receive_upstream(fulfilled_to_factory)

fulfilled_to_regional_warehouse = factory_agent.give_downstream(regional_warehouse_demand)
regional_warehouse_agent.receive_upstream(fulfilled_to_regional_warehouse)

fulfilled_to_wholesale = regional_warehouse_agent.give_downstream(wholesale_demand)
wholesale_agent.receive_upstream(fulfilled_to_wholesale)

fulfilled_to_retail = wholesale_agent.give_downstream(retail_demand)
retail_agent.receive_upstream(fulfilled_to_retail)

fulfilled_to_customer = retail_agent.give_downstream(customer_demand)

In [ ]:
print('Retail agent asked for ' + str(retail_demand) + ' and received ' + str(fulfilled_to_retail))
print('Wholesale agent asked for ' + str(wholesale_demand) + ' and received ' + str(fulfilled_to_wholesale))
print('Regional Warehouse agent asked for ' + str(regional_warehouse_demand) + ' and received ' + str(fulfilled_to_regional_warehouse))
print('Factory agent asked for ' + str(factory_demand) + ' and received ' + str(fulfilled_to_factory))

# Policy Iteration

This is still work in progress

In [ ]:

total_epochs = 10000


# TODO create a function that doesn't learn, only
# asks on t for what the downstream agent asked for on t-1
def order_by_the_day(agent,day):
    return agent.downstream_agent.current_policy[day]

def create_demand(day):
    x = np.random.uniform(0, 1)
    if x < p_exploration:  # exploRation
        return random.randint(0,1000)  # TODO check if is 1,000 a good number?
    else:  # exploRation
        return agent.best_policy(day)

# POLICY ITERATION  ----------------------------------------------------------------  
    
for j in range(total_epochs):  # 10000 epochs, need to find a better way to constraint
    p_exploration = (total_epochs - j) / total_epochs  # starts in 1 ends in 0
    for day in range(365):  # one year
        for agent in agents:
            # PART 1
            # Transactions for previous day happen. These are fixed.
            # Paying for warehousing overnight
            agent.pay_for_warehousing
            # Orders are fulfilled first time in the morning
            # TODO: WRAP THESE TRANSACTIONS INTO THE AGENT CLASS - somehow predetermine the upstream and downstream actors
            # Everyone gets their shippings at the same time
            # Factory
            fulfilled_to_factory = field_supply
            factory_agent.receive_upstream(fulfilled_to_factory)
            # Regional Warehouse
            fulfilled_to_regional_warehouse = factory_agent.give_downstream(regional_warehouse_demand)
            regional_warehouse_agent.receive_upstream(fulfilled_to_regional_warehouse)
            # Wholesale
            fulfilled_to_wholesale = regional_warehouse_agent.give_downstream(wholesale_demand)
            wholesale_agent.receive_upstream(fulfilled_to_wholesale)
            # Retail
            fulfilled_to_retail = wholesale_agent.give_downstream(retail_demand)
            retail_agent.receive_upstream(fulfilled_to_retail)
            # Customer
            fulfilled_to_customer = retail_agent.give_downstream(customer_demand)
            # How much money did the agent end up with yesterday's decisions?
            agent.current_payout.append(agent.total_money)
            # PART 2
            # Agent decides demand for today, which will (might) be fulfilled tomorrow
            agent_demand = create_demand(day)
            agent.current_policy.append(agent_demand)
        if agent.current_payout[-1] > agent.best_payout[-1]:  # payout at the end of the year
            agent.best_policy = agent.current_policy
            agent.best_payout = agent.current_payout